In [1]:
import mysql.connector
import pandas as pd
from IPython.display import Markdown, display

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="secret",
  port="3306",
  database="serlo"
)

def printmd(string):
    display(Markdown(string))

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def query_number(sql):
    return query(sql)[0][0]

In [2]:
year = 2022

printmd(f"# Auswertung für das Jahr {year}")

# Auswertung für das Jahr 2022

In [3]:
count = query_number("select count(*) from event_log where YEAR(event_log.date) = %s and event_log.event_id IN (5);" % year)

printmd(f"#### Anzahl Bearbeitungen: {count}")

#### Anzahl Bearbeitungen: 17495

In [4]:
new_count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name = "article" and year(entity.date) = %s
""" % year)
count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name = "article"
""")

printmd(f"#### Anzahl neuer Artikel in diesem Jahr: {new_count}")
printmd(f"#### Aktueller Bestand von Artikel: {count}")

#### Anzahl neuer Artikel in diesem Jahr: 518

#### Aktueller Bestand von Artikel: 4037

In [5]:
new_count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name = "course-page" and year(entity.date) = %s
""" % year)
count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name = "course-page"
""")

printmd(f"#### Anzahl neuer Kursseiten in diesem Jahr: {new_count}")
printmd(f"#### Aktueller Bestand von Kursen: {count}")

#### Anzahl neuer Kursseiten in diesem Jahr: 429

#### Aktueller Bestand von Kursen: 4560

In [6]:
new_count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name in ("text-exercise", "grouped-text-exercise") and year(entity.date) = %s
""" % year)
count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name in ("text-exercise", "grouped-text-exercise")
""")

printmd(f"#### Anzahl neuer Aufgaben in diesem Jahr: {new_count}")
printmd(f"#### Aktueller Bestand von Aufgaben: {count}")

#### Anzahl neuer Aufgaben in diesem Jahr: 1926

#### Aktueller Bestand von Aufgaben: 16575

In [7]:
new_count = query_number("""
    select count(*) from comment
    join uuid on comment.id = uuid.id
    where uuid.trashed = 0 and year(comment.date) = %s
""" % year)
count = query_number("""
    select count(*) from comment
    join uuid on comment.id = uuid.id
    where uuid.trashed = 0
""")

printmd(f"#### Anzahl neuer Kommentare in diesem Jahr: {new_count}")
printmd(f"#### Aktueller Bestand von Kommentaren: {count}")

#### Anzahl neuer Kommentare in diesem Jahr: 749

#### Aktueller Bestand von Kommentaren: 9669

In [8]:
printmd("### Liste von Autor:innen der Anzahl ihrer Bearbeitung Bearbeitungen")
pd.set_option('display.max_rows', None)
df = pd.read_sql("""
    select user.username, count(event_log.actor_id) as edits from event_log
    join user on user.id = event_log.actor_id
    where year(event_log.date) = %s
    group by event_log.actor_id
    order by edits desc
""" % year, db)
df.to_csv("/tmp/authors.csv")
df

### Liste von Autor:innen der Anzahl ihrer Bearbeitung Bearbeitungen

<ipython-input-8-c4b68e3323b1>:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


,username,edits
0,Felix_Eccardt,11199
1,Karin,5464
2,Kowalsky,4970
3,Sascha_Lill_95,3892
4,LinaMaria,3603
5,Peter,2901
6,metzgaria,2429
7,kathongi,1931
8,Habiba,1770
9,korbi_d,1619
